## Bayesian Sequential Testing
## Introduction


This document outlines a procedure for conducting A/B/n Bayesian testing based on the approach described in [VWO's 
white paper](https://cdn2.hubspot.net/hubfs/310840/VWO_SmartStats_technical_whitepaper.pdf).
 The Python implementation is available in [here](https://stash.aexp.com/stash/projects/ONE-STREAM/repos/xp-algorithms/browse/sequential-testing/vwo).  
  
A binary-outcome test (e.g. a test of conversion rates) can be modeled as a binomial sequence with parameter $p$. The beta distribution is a convenient prior distribution for modeling a binomial parameter $p$. Starting from a non-informative prior $Beta\left(1, 1\right)$, the posterior probability distribution of $p$ after $N$ events with $S$ successes is given by $Beta\left(S+1,N-S+1\right)$ accoring to Bayes theorem. Therefore, throughout the test, we only need to track only 2 numbers to compute the posterior distribution - $N$ an $S$.

Once we can derive the probability distribution for a test, we can easily compute many useful statistics. 
For example, we can compute the probability that the variant $A$ is better than the variant $B$ using the joint
probability distrbution.

$$X:\,Random\,variable\,for\:conversion\,rate\,of\,variant\,A\,$$
$$\,Y:\,Random\,variable\,for\,conversion\,rate\,of\,variant\,B$$

$$X\sim Beta\left(a,b\right),Y\sim Beta\left(c,d\right)$$

$$P\left(X>Y\right)= \int_0^1\int_y^1 \frac{x^a\left(1-x\right)^b}{B\left(a,b\right)} \frac{y^c\left(1-y\right)^d}{B\left(c,d\right)}~dx~dy 
\equiv g\left(a, b, c, d\right)
= \frac{\Gamma\left(d+b\right)\Gamma\left(d+c\right)}{\Gamma\left(d+b+c\right)\Gamma\left(d\right)} + \sum_{i=1}^{a-1} \left(\frac{\Gamma\left(i+c\right)\Gamma\left(b+d\right)\Gamma\left(i+b\right)\Gamma\left(c+d\right)}{\Gamma\left(i\right)\Gamma\left(b\right)\Gamma\left(c\right)\Gamma\left(d\right)\Gamma\left(i+b+c+d\right)}\right) / i$$
   
$$B\left(a, b\right)\,is\,the\,beta\,fuction\,with\,parameters\,a\,and\,b$$
$$\Gamma\left(a\right)\,is\,the\,gamma\,fuction\,with\,parameter\,a$$
   
Note:  The derivation of the above closed-form formula can be found [here](https://www.johndcook.com/UTMDABTR-005-05.pdf). 

Here is VWO's dashboard for reporting test results which includes many additonal statistics.

<img src="https://static.wingify.com/vwo/wp-content/themes/vwo/images/smartstats/smart-stat-report-table.png">

Source: https://vwo.com/resources/bayesian-ab-testing/

These statistics can be reproduced by the following code:

In [1]:
import util.el_core as el_core
el_core.calc_stat(sa=250, na=11221, sb=1600, nb=14187, relative_mde_value=0.02)

{'data': [250, 11221, 1600, 14187],
 'beta_param': [251, 10972, 1601, 12588],
 'cvr_a': 0.022364786598948586,
 'cvr_b': 0.11283388540418635,
 'cvr_a_ci': [0.01892988453955432, 0.02611928740825431],
 'cvr_b_ci': [0.10609539604453345, 0.11977736047524634],
 'prob_b_gt_a': 1.0,
 'lift': 0.09046909880523776,
 'el_a': 0.0904690988100059,
 'el_b': 8.835834986583339e-194,
 'el_res': 'B'}

## Running a Bayesian sequential test
## A/B Case

In [2]:
import util.helper as helper
ss = helper.sample_size(alpha=0.05, beta=0.05, mu=0.2, relative_mde_value=0.02, variants=2, pr=True)

Sample size for mu=0.2000, relative_mde_value=0.0200, absolute_mde_value=0.0040, alpha=0.0500, beta=0.0500:
variant_sample_size=261,830, total_sample_size=523,661


VWO define the loss function and expected loss as below. Expected loss represents the expected losses associated with 
choosing "A" or "B" as the winner. 

**Loss function:**
  
$L_x=max\left(Y-X, 0\right),L_y=max\left(X-Y, 0\right)$

$$E[L_x] = \int_0^1\int_0^1 max\left(y-x,0\right)\frac{x^a\left(1-x\right)^b}{B\left(a,b\right)} \frac{y^c\left(1-y\right)^d}{B\left(c,d\right)}~dx~dy
= \frac{B\left(c+1,d\right)}{B\left(c,d\right)}g\left(c+1,d,a,b\right) - 
\frac{B\left(a+1,b\right)}{B\left(a,b\right)}g\left(c,d,a+1,b\right)$$

$$E[L_y] = \int_0^1\int_0^1 max\left(x-y,0\right)\frac{x^a\left(1-x\right)^b}{B\left(a,b\right)} \frac{y^c\left(1-y\right)^d}{B\left(c,d\right)}~dx~dy
= \frac{B\left(a+1,b\right)}{B\left(a,b\right)}g\left(a+1,b,c,d\right) - 
\frac{B\left(c+1,d\right)}{B\left(c,d\right)}g\left(a,b,c+1,d\right)$$
  
  
Note: the derivation of the above closed-form formula can be found [here](https://www.chrisstucchio.com/blog/2014/bayesian_ab_decision_rule.html).

**Decision rule:**  
  
Chose a threshold of caring($toc$). If the expected loss of a variant is below $toc$, delare this variant as the winner.

Based on the simulation study, [threshold of caring values recommended by VWO](https://vwo.com/knowledge/what-is-smart-decision) normally results in high type 1 error rates(~12% for high certainty values). So more conversative toc values are used. A toc calculation example is show below:

In [3]:
# 0.00000250(the value after sequential:) is the suggested toc for running a sequential test.
import util.helper as helper
toc = helper.estimate_toc(mu=0.2, relative_mde_value=0.02, pr=True)

TOC for mu=0.2000, relative_mde_value=0.0200, absolute_mde_value=0.0040: toc_adj_factor:0.00025:
VWO recommended values: high certainty: 0.00004000, balance: 0.00030000, quick learning: 0.00080000
Suggested value for One XP: 0.00000100


A simulated example based on the suggested $toc$:

In [4]:
import numpy as np
import util.el_core as el_core
np.random.seed(2016)
data = [np.random.binomial(1, 0.2, ss['variant_sample_size']), np.random.binomial(1, 0.2, ss['variant_sample_size'])]
el_core.calc_ab(data, relative_mde_value=0.02) # plug the data into calc_ab to produce output statistics for a test

{'data': [52344, 261830, 52559, 261830],
 'beta_param': [52345, 209487, 52560, 209272],
 'cvr_a': 0.19991826820251155,
 'cvr_b': 0.2007394054202695,
 'cvr_a_ci': [0.19790932795232713, 0.20193581917646042],
 'cvr_b_ci': [0.19872735855654716, 0.2027600394456196],
 'prob_b_gt_a': 0.7710507481834813,
 'lift': 0.0008211372177579501,
 'el_a': 0.0009681703411913711,
 'el_b': 0.00014703305359205915,
 'el_res': 'U'}

In order to control type 1 error rates, we need to use even more conservative $toc$ for sequential testing. 
For given $p_0$ and $mde$, the suggested toc is $p_0*mde*0.00025$. The simulation study shows that this suggested 
value can limit the type 1 error rates under 5% after 1,000 peeks.

## Simulation

In [1]:
import util.helper as helper
import util.sim_el as sim_el
ss = helper.sample_size(mu=0.2, relative_mde_value=0.02, pr=True)

Sample size for mu=0.2000, relative_mde_value=0.0200, absolute_mde_value=0.0040, alpha=0.0500, beta=0.0500:
variant_sample_size=261,830, total_sample_size=523,661


In [2]:
sim_data = sim_el.sim_peeking(muA=0.2, muB=0.2, sample_size=ss['variant_sample_size'], 
                              n_experiments=1000, relative_mde_value=0.02, n_peeks=25)

[Start time]: 2020-02-03 23:04:40
[Parameters]: muA: 0.2, muB: 0.2, sample_size: 261,830, n_experiments: 1,000, relative_mde_value: 0.02, toc_adj_factor: 0.00025, n_peeks: 25, start: 10,473, step: 10,473
[02-03 23:04:51]: Peek #1 @ 10,473 samples, avg_lift: 0.01833, el_res: {'U': 999, 'A': 1}
[02-03 23:04:54]: Peek #2 @ 20,946 samples, avg_lift: 0.01313, el_res: {'U': 996, 'A': 2, 'B': 2}
[02-03 23:04:58]: Peek #3 @ 31,419 samples, avg_lift: 0.00980, el_res: {'U': 995, 'B': 3, 'A': 2}
[02-03 23:05:03]: Peek #4 @ 41,892 samples, avg_lift: 0.00850, el_res: {'U': 993, 'A': 4, 'B': 3}


/Users/thuan/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/thuan/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[02-03 23:05:08]: Peek #5 @ 52,365 samples, avg_lift: nan, el_res: {'U': 993, 'A': 4, 'B': 3}
[02-03 23:05:15]: Peek #6 @ 62,838 samples, avg_lift: 0.00719, el_res: {'U': 992, 'A': 5, 'B': 3}
[02-03 23:05:23]: Peek #7 @ 73,311 samples, avg_lift: 0.00641, el_res: {'U': 989, 'A': 7, 'B': 4}
[02-03 23:05:31]: Peek #8 @ 83,784 samples, avg_lift: 0.00586, el_res: {'U': 988, 'A': 8, 'B': 4}
[02-03 23:05:41]: Peek #9 @ 94,257 samples, avg_lift: 0.00541, el_res: {'U': 987, 'A': 9, 'B': 4}
[02-03 23:05:52]: Peek #10 @ 104,730 samples, avg_lift: 0.00537, el_res: {'U': 985, 'A': 11, 'B': 4}
[02-03 23:06:03]: Peek #11 @ 115,203 samples, avg_lift: nan, el_res: {'U': 985, 'A': 11, 'B': 4}
[02-03 23:06:16]: Peek #12 @ 125,676 samples, avg_lift: 0.00478, el_res: {'U': 984, 'A': 11, 'B': 5}
[02-03 23:06:30]: Peek #13 @ 136,149 samples, avg_lift: nan, el_res: {'U': 984, 'A': 11, 'B': 5}
[02-03 23:06:44]: Peek #14 @ 146,622 samples, avg_lift: nan, el_res: {'U': 984, 'A': 11, 'B': 5}
[02-03 23:07:00]: Pee

The EL decision rule needs an average sample size of 158,379 to get 95% power compared to 261,000 sample size needed for T-test.

In [3]:
sim_data = sim_el.sim_peeking(muA=0.2, muB=0.204, sample_size=int(ss['variant_sample_size']*1.45), 
                              n_experiments=1000, relative_mde_value=0.02, n_peeks=25)

[Start time]: 2020-02-03 23:13:25
[Parameters]: muA: 0.2, muB: 0.204, sample_size: 379,653, n_experiments: 1,000, relative_mde_value: 0.02, toc_adj_factor: 0.00025, n_peeks: 25, start: 15,186, step: 15,186
[02-03 23:13:40]: Peek #1 @ 15,186 samples, avg_lift: 0.01600, el_res: {'U': 987, 'B': 13}
[02-03 23:13:44]: Peek #2 @ 30,372 samples, avg_lift: 0.01087, el_res: {'U': 955, 'B': 45}
[02-03 23:13:49]: Peek #3 @ 45,558 samples, avg_lift: 0.00900, el_res: {'U': 915, 'B': 85}
[02-03 23:13:55]: Peek #4 @ 60,744 samples, avg_lift: 0.00738, el_res: {'U': 865, 'B': 135}
[02-03 23:14:02]: Peek #5 @ 75,930 samples, avg_lift: 0.00664, el_res: {'U': 810, 'B': 190}
[02-03 23:14:10]: Peek #6 @ 91,116 samples, avg_lift: 0.00606, el_res: {'U': 748, 'B': 252}
[02-03 23:14:18]: Peek #7 @ 106,302 samples, avg_lift: 0.00536, el_res: {'U': 685, 'B': 315}
[02-03 23:14:26]: Peek #8 @ 121,488 samples, avg_lift: 0.00500, el_res: {'U': 623, 'B': 377}
[02-03 23:14:35]: Peek #9 @ 136,674 samples, avg_lift: 0.00

## References:

[1] C. Stucchio, Bayesian A/B Testing at VWO, Whitepaper, Visual Website Optimizer, 2015.